In [9]:
import tensorflow as tf
from tensorflow import image as img
import os
import shutil, sys
import numpy as np
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import random
import keras
from keras.utils import plot_model
from keras.applications.mobilenet import MobileNet
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from keras.layers import Dropout, Flatten, Dense, Input
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

In [10]:
from keras.models import load_model
model = load_model('keras_aug_4_15_12.h5')

In [3]:
input = Input(shape=(196,196,3))
x = model.layers[0].get_layer("block1_conv1") (input)
x = model.layers[0].get_layer("block1_conv2") (x)
x = model.layers[0].get_layer("block1_pool") (x)
x = model.layers[0].get_layer("block2_conv1") (x)
x = model.layers[0].get_layer("block2_conv2") (x)
x = model.layers[0].get_layer("block2_pool") (x)
x = model.layers[0].get_layer("block3_conv1") (x)
x = model.layers[0].get_layer("block3_conv2") (x)
x = model.layers[0].get_layer("block3_conv3") (x)
x = model.layers[0].get_layer("block3_conv4") (x)
x = model.layers[0].get_layer("block3_pool") (x)
x = model.layers[0].get_layer("block4_conv1") (x)
x = model.layers[0].get_layer("block4_conv2") (x)
x = model.layers[0].get_layer("block4_conv3") (x)
x = model.layers[0].get_layer("block4_conv4") (x)
x = model.layers[0].get_layer("block4_pool") (x)
x = model.layers[0].get_layer("block5_conv1") (x)
x = model.layers[0].get_layer("block5_conv2") (x)
x = model.layers[0].get_layer("block5_conv3") (x)
x = model.layers[0].get_layer("block5_conv4") (x)
x = model.layers[0].get_layer("block5_pool") (x)
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.3) (x)
x = Dense(256, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
x = Dense(32, activation = "relu")(x)
x = Dense(2, activation='softmax') (x)
model= Model(input, x)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 196, 196, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 196, 196, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 196, 196, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 98, 98, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 98, 98, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 98, 98, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 49, 49, 128)       0         
__________

In [11]:
input = Input(shape=(196,196,3))
x = model.get_layer("vgg19")(input)
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.3) (x)
x = Dense(256, activation = "relu")(x)
x = Dense(64, activation = "relu")(x)
x = Dense(32, activation = "relu")(x)
preds = Dense(2, activation='softmax') (x)

model= Model(input, preds)

for layer in model.layers:
    if layer.name == 'vgg19':
        layer.trainable = False
model.compile(optimizer=optimizers.RMSprop(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 196, 196, 3)       0         
_________________________________________________________________
vgg19 (Model)                (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten_3 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              18875392  
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               262400    
_________________________________________________________________
dense_13 (Dense)             (None, 64)                16448     
__________

In [8]:
image_size = 196
batch_size = 30
train2_dir = './train2'
val2_dir = './validation2'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=360,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip = True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator_2 = train_datagen.flow_from_directory(
                    train2_dir,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator_2 = validation_datagen.flow_from_directory(
                        val2_dir,
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 857 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


In [8]:
class_weight = {0: 2.67,
                1: 1.0}

In [9]:
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 26s - loss: 0.9496 - acc: 0.6096 - val_loss: 0.5498 - val_acc: 0.7500
Epoch 2/10
 - 23s - loss: 0.8032 - acc: 0.8077 - val_loss: 0.4482 - val_acc: 0.7833
Epoch 3/10
 - 24s - loss: 0.6836 - acc: 0.8388 - val_loss: 0.3889 - val_acc: 0.8667
Epoch 4/10
 - 23s - loss: 0.5658 - acc: 0.8632 - val_loss: 0.3509 - val_acc: 0.8500
Epoch 5/10
 - 24s - loss: 0.5063 - acc: 0.8864 - val_loss: 0.3239 - val_acc: 0.8500
Epoch 6/10
 - 24s - loss: 0.4748 - acc: 0.8833 - val_loss: 0.2972 - val_acc: 0.8500
Epoch 7/10
 - 24s - loss: 0.4010 - acc: 0.9037 - val_loss: 0.3538 - val_acc: 0.8833
Epoch 8/10
 - 24s - loss: 0.3953 - acc: 0.9040 - val_loss: 0.3399 - val_acc: 0.8667
Epoch 9/10
 - 24s - loss: 0.3698 - acc: 0.9046 - val_loss: 0.3320 - val_acc: 0.8500
Epoch 10/10
 - 23s - loss: 0.3558 - acc: 0.8960 - val_loss: 0.4118 - val_acc: 0.8667


In [10]:
for layer in model.layers:
    if layer.name == 'vgg19':
        layer.trainable = True
model.compile(optimizer=optimizers.RMSprop(lr=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 26s - loss: 0.5563 - acc: 0.8445 - val_loss: 0.1998 - val_acc: 0.9333
Epoch 2/10
 - 24s - loss: 0.4284 - acc: 0.8764 - val_loss: 0.9136 - val_acc: 0.7667
Epoch 3/10
 - 24s - loss: 0.4337 - acc: 0.8850 - val_loss: 0.5915 - val_acc: 0.8000
Epoch 4/10
 - 23s - loss: 0.4154 - acc: 0.8902 - val_loss: 0.4810 - val_acc: 0.7667
Epoch 5/10
 - 23s - loss: 0.4547 - acc: 0.8657 - val_loss: 0.3365 - val_acc: 0.8833
Epoch 6/10
 - 24s - loss: 0.3971 - acc: 0.8770 - val_loss: 0.5552 - val_acc: 0.8667
Epoch 7/10
 - 24s - loss: 0.3965 - acc: 0.8933 - val_loss: 0.4600 - val_acc: 0.8667
Epoch 8/10
 - 25s - loss: 0.4831 - acc: 0.8680 - val_loss: 0.3473 - val_acc: 0.8667
Epoch 9/10
 - 24s - loss: 0.3671 - acc: 0.9017 - val_loss: 0.2362 - val_acc: 0.9167
Epoch 10/10
 - 23s - loss: 0.3973 - acc: 0.8773 - val_loss: 0.2859 - val_acc: 0.9000


In [11]:
model.compile(optimizer=optimizers.RMSprop(lr=0.000005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.3588 - acc: 0.8922 - val_loss: 0.2527 - val_acc: 0.8833
Epoch 2/10
 - 24s - loss: 0.3083 - acc: 0.9150 - val_loss: 0.2530 - val_acc: 0.9167
Epoch 3/10
 - 24s - loss: 0.2791 - acc: 0.9259 - val_loss: 0.5165 - val_acc: 0.8333
Epoch 4/10
 - 24s - loss: 0.2974 - acc: 0.9172 - val_loss: 0.3037 - val_acc: 0.8833
Epoch 5/10
 - 24s - loss: 0.3042 - acc: 0.9206 - val_loss: 0.3283 - val_acc: 0.9333
Epoch 6/10
 - 24s - loss: 0.2992 - acc: 0.9163 - val_loss: 0.7285 - val_acc: 0.7667
Epoch 7/10
 - 24s - loss: 0.2409 - acc: 0.9379 - val_loss: 0.2294 - val_acc: 0.9500
Epoch 8/10
 - 23s - loss: 0.2519 - acc: 0.9252 - val_loss: 0.7367 - val_acc: 0.8167
Epoch 9/10
 - 24s - loss: 0.2694 - acc: 0.9190 - val_loss: 0.4804 - val_acc: 0.8667
Epoch 10/10
 - 23s - loss: 0.2680 - acc: 0.9141 - val_loss: 0.1549 - val_acc: 0.9167


In [12]:
model.compile(optimizer=optimizers.RMSprop(lr=0.000001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.2276 - acc: 0.9405 - val_loss: 0.3326 - val_acc: 0.8833
Epoch 2/10
 - 23s - loss: 0.2295 - acc: 0.9370 - val_loss: 0.3328 - val_acc: 0.9167
Epoch 3/10
 - 23s - loss: 0.1935 - acc: 0.9436 - val_loss: 0.3208 - val_acc: 0.9000
Epoch 4/10
 - 23s - loss: 0.2124 - acc: 0.9428 - val_loss: 0.3775 - val_acc: 0.8833
Epoch 5/10
 - 23s - loss: 0.1788 - acc: 0.9528 - val_loss: 0.3424 - val_acc: 0.9167
Epoch 6/10
 - 24s - loss: 0.2011 - acc: 0.9546 - val_loss: 0.3135 - val_acc: 0.9000
Epoch 7/10
 - 23s - loss: 0.2255 - acc: 0.9370 - val_loss: 0.2784 - val_acc: 0.9333
Epoch 8/10
 - 24s - loss: 0.1615 - acc: 0.9647 - val_loss: 0.5471 - val_acc: 0.8667
Epoch 9/10
 - 23s - loss: 0.2337 - acc: 0.9368 - val_loss: 0.2198 - val_acc: 0.9333
Epoch 10/10
 - 23s - loss: 0.1990 - acc: 0.9517 - val_loss: 0.2599 - val_acc: 0.9167


In [13]:
model.save('2class_best_1.h5')

In [14]:
model.compile(optimizer=optimizers.RMSprop(lr=0.0000005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history_2 = model.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(857/batch_size)),
                    epochs=10,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(60/batch_size)),
                    verbose=2,
                    class_weight=class_weight)

Epoch 1/10
 - 25s - loss: 0.2409 - acc: 0.9356 - val_loss: 0.2503 - val_acc: 0.9333
Epoch 2/10
 - 24s - loss: 0.1772 - acc: 0.9543 - val_loss: 0.4239 - val_acc: 0.8833
Epoch 3/10
 - 24s - loss: 0.1747 - acc: 0.9520 - val_loss: 0.4446 - val_acc: 0.9000
Epoch 4/10
 - 23s - loss: 0.2079 - acc: 0.9486 - val_loss: 0.2387 - val_acc: 0.9333
Epoch 5/10
 - 25s - loss: 0.1669 - acc: 0.9505 - val_loss: 0.3670 - val_acc: 0.8833
Epoch 6/10
 - 22s - loss: 0.1875 - acc: 0.9535 - val_loss: 0.3536 - val_acc: 0.9167
Epoch 7/10
 - 23s - loss: 0.1960 - acc: 0.9477 - val_loss: 0.5908 - val_acc: 0.8667
Epoch 8/10
 - 25s - loss: 0.1931 - acc: 0.9420 - val_loss: 0.2998 - val_acc: 0.9333
Epoch 9/10
 - 23s - loss: 0.1989 - acc: 0.9413 - val_loss: 0.3460 - val_acc: 0.9167
Epoch 10/10
 - 24s - loss: 0.1664 - acc: 0.9471 - val_loss: 0.5670 - val_acc: 0.8667


In [3]:
from keras.models import load_model
model = load_model('2class_best_1.h5')

In [4]:
# Save the weights
model.save_weights('2class_model_weights.h5')

# Save the model architecture
with open('2class_model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [8]:
val2_dir = './validation2'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        val2_dir,
        target_size=(196, 196),
        color_mode="rgb",
        shuffle = False,
        class_mode='categorical',
        batch_size=1)

filenames = test_generator.filenames
nb_samples = len(filenames)

predict = model.predict_generator(test_generator,steps = nb_samples)

Found 60 images belonging to 2 classes.


In [9]:
predict_labels = np.argmax(predict, axis=1)
val_labels = np.zeros(predict_labels.size, dtype=int)
for i in range(2):
    for j in range(30):
        val_labels[i*30+j] = int(i);

In [10]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(val_labels, predict_labels)

array([[25,  5],
       [ 0, 30]])